In [1]:
import pandas as pd
import numpy as np
import json
import seaborn as sns
from sklearn.metrics import balanced_accuracy_score


Bad key text.latex.preview in file /home/cherepaha/miniconda/envs/py37/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 123 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file /home/cherepaha/miniconda/envs/py37/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 155 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.jpeg_quality in file /home/cherepaha/miniconda/envs/py37/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 418 ('savefig.jpeg_quality: 95       # w

In [2]:
all_meta = pd.read_csv('data/train_metadata_extended.csv')
with open('data/scored_birds.json') as fin:
    test_birds = json.load(fin)

In [3]:
all_meta.loc[:, 'secondary_labels'] = all_meta.secondary_labels.apply(eval)
all_meta['target_raw'] = all_meta.secondary_labels + all_meta.primary_label.apply(lambda x: [x])

In [4]:
all_species = sorted(set(all_meta.target_raw.sum()))
species2id = {s: i for i, s in enumerate(all_species)}
id2species = {i: s for i, s in enumerate(all_species)}

all_meta['target'] = all_meta.target_raw.apply(lambda species: [int(s in species) for s in all_species])

In [5]:
last_pred = np.load('Lab02_NMT/last_pred.npy')
last_true = np.load('Lab02_NMT/last_true.npy')

In [6]:
test_idx = [species2id[b] for b in test_birds]
last_true[:, test_idx].sum(axis=0)

array([  4.,   6.,  25.,   3.,   0.,   7.,   1.,  13.,   0.,   2.,   1.,
         0.,  87.,  16.,  18.,   0.,   9.,   0., 107.,  22.,  10.],
      dtype=float32)

In [7]:
def score_pred(true, pred):
    if true.sum() == 0:
        return np.nan
    return balanced_accuracy_score(
            true,
            pred
        )

In [8]:
score_stat = []
trsh = [0.01, 0.05, 0.1, 0.15, 0.2, 0.25]

for t in trsh:
    score_stat.append({
        b: score_pred(
            last_true[:, species2id[b]],
            last_pred[:, species2id[b]] > t
        )
        for b in test_birds
    })
    
score_stat = pd.DataFrame(score_stat).T
score_stat.columns = trsh

In [9]:
score_stat

,0.01,0.05,0.10,0.15,0.20,0.25
akiapo,0.874326,0.750000,0.750000,0.750000,0.750000,0.625000
aniani,0.749494,0.750000,0.750000,0.666667,0.666667,0.583333
apapan,0.973720,0.857284,0.857963,0.838642,0.838982,0.819151
barpet,0.833165,0.833165,0.833333,0.833333,0.833333,0.833333
crehon,NaN,NaN,NaN,NaN,NaN,NaN
elepai,0.714286,0.714286,0.714286,0.714286,0.714286,0.642857
ercfra,1.000000,0.500000,0.500000,0.500000,0.500000,0.500000
hawama,0.915132,0.803805,0.805664,0.729417,0.691463,0.691632
hawcre,NaN,NaN,NaN,NaN,NaN,NaN
hawgoo,0.998316,0.749832,0.750000,0.750000,0.750000,0.750000


In [13]:
score_stat.idxmax(axis=1).fillna().to_dict()

{'akiapo': 0.01,
 'aniani': 0.05,
 'apapan': 0.01,
 'barpet': 0.1,
 'crehon': nan,
 'elepai': 0.01,
 'ercfra': 0.01,
 'hawama': 0.01,
 'hawcre': nan,
 'hawgoo': 0.01,
 'hawhaw': 0.1,
 'hawpet1': nan,
 'houfin': 0.01,
 'iiwi': 0.01,
 'jabwar': 0.05,
 'maupar': nan,
 'omao': 0.01,
 'puaioh': nan,
 'skylar': 0.01,
 'warwhe1': 0.01,
 'yefcan': 0.1}

In [14]:
with open('Lab02_NMT/model_save/baseline_39fvwny4/trsh.json', 'w') as fout:
    json.dump(score_stat.idxmax(axis=1).fillna(0.05).to_dict(), fout)